In [1]:
import pandas as pd
import os
from keras.models import Sequential
from keras.layers.core import Dense, Activation
model = Sequential()
path = "/Users/petronillagriffith/Desktop/test"

filename = os.path.join(path,"SN_d_tot_V2.0.csv")
names = ['year', 'month', 'day', 'dec_year', 'sn_value', 'sn_error', 'obs_num']
df = pd.read_csv(filename, sep= ';' ,header=None, names=names, na_values = ['-1'], index_col = False)

print("Starting file:")
print(df[20:100])

print("Ending file:")
print(df[-10:])

Using TensorFlow backend.


FileNotFoundError: File b'/Users/petronillagriffith/Desktop/test/DATA/SN_d_tot_V2.0.csv' does not exist

In [9]:
df = df[df.sn_error.notnull()]
df

,year,month,day,dec_year,sn_value,sn_error,obs_num
7,1818,1,8,1818.021,65,10.2,1
12,1818,1,13,1818.034,37,7.7,1
16,1818,1,17,1818.045,77,11.1,1
17,1818,1,18,1818.048,98,12.6,1
18,1818,1,19,1818.051,105,13.0,1
24,1818,1,25,1818.067,25,6.3,1
27,1818,1,28,1818.075,38,7.8,1
28,1818,1,29,1818.078,20,5.7,1
33,1818,2,3,1818.092,17,5.2,1
35,1818,2,5,1818.097,20,5.7,1


In [10]:
len(df)

69953

In [11]:
df.dtypes

year          int64
month         int64
day           int64
dec_year    float64
sn_value      int64
sn_error    float64
obs_num       int64
dtype: object

In [14]:
def missing_median(df, names):
    med = df[names].median()
    df[names] = df[names].fillna(med)

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [15]:
score = metrics.mean_squared_error(y)
print("Score (MSE): {}".format(score))

NameError: name 'metrics' is not defined

In [20]:
model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(50, activation='relu')) # Hidden 2
model.add(Dense(25)) # Output
model.add(Dense(12))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=2,epochs=10000)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [17]:
pred = model.predict(x)
score = np.sqrt(metrics.mean_squared_error(pred, y))
print("Score (RMSE): {}".format(score))

NameError: name 'x' is not defined

In [2]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
import numpy as np

print('Build model...')
model = Sequential()
model.add(Dense(64,input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=le-3, patience=5, verbose=1, mode='auto')
print('Train...')

model.fit(x_train,y_train,validation_data=(x_test,y_test), callbacks=[monitor], verbose=2, epochs=1000)

Using TensorFlow backend.


Build model...


NameError: name 'x_train' is not defined